## Still under development

In [11]:
# See install in M2-8a

In [1]:
from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()
# Now you can access the environment variables
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')
langchain_api_key = os.getenv('LANGCHAIN_API_KEY')

In [2]:
from langchain_core.prompts import ChatPromptTemplate

In [3]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [23]:
#%pip install langchain_openai
#%pip show langchain_openai
#%pip show typing-extensions

Name: langchain-openai
Version: 0.1.3
Summary: An integration package connecting OpenAI and LangChain
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /opt/conda/lib/python3.10/site-packages
Requires: langchain-core, openai, tiktoken
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [4]:
# %pip uninstall typing_extensions -y
# %pip install typing_extensions
#%pip install --force-reinstall typing-extensions==4.5
#pip install --force-reinstall openai==1.8

In [9]:

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=openai_api_key)

In [4]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(api_key=openai_api_key)

In [5]:
# Vector store
#%pip install faiss-cpu

In [6]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [7]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [21]:
# # Create a model
# from langchain_openai import ChatOpenAI
# print(openai_api_key)
# llm = ChatOpenAI(api_key="openai_api_key")
# llm.invoke("how can langsmith help with testing?")

In [10]:
from langchain.chains.combine_documents import create_stuff_documents_chain
#from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [11]:
# Just local context
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by allowing users to visualize test results.'

In [12]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [13]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmith can help with testing by enabling developers to create datasets, run tests on LLM applications, upload test cases, run custom evaluations, compare different versions of applications, use a playground environment for rapid iteration, beta test to collect real-world performance data, capture feedback from users, annotate traces for evaluation criteria, add runs to datasets for refining performance, monitor key metrics over time, conduct A/B testing, set up automations for processing traces in real-time, and track performance across multiple turns in multi-turn applications.
